In [1]:
!pip install geopy

     |████████████████████████████████| 122kB 5.0MB/s eta 0:00:01


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
!pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 2.6MB/s ta 0:00:011
     |████████████████████████████████| 92kB 4.3MB/s eta 0:00:011
     |████████████████████████████████| 829kB 6.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [8]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup

     |████████████████████████████████| 122kB 6.3MB/s eta 0:00:01


end part 1

**This marks the end of the libraries imported** 
After this we would use the beautiful soup package inorder to extract data from internet.

In [9]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_communities_and_neighborhoods_of_San_Diego").text

In [10]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [11]:
neighborhoodList = []
for row in soup.find_all("table", class_="multicol")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [ ]:
san_df = pd.DataFrame({"Neighborhood": neighborhoodList})

In [13]:
arr = [16,36,53,73,90,99]
for i in arr:
    san_df.drop(i,inplace = True)
san_df.reset_index(inplace = True, drop = True)

In [10]:
san_df.to_csv('SanDiego_neighborhood.csv')

**The code below is to extract the location co-ordinates of a place**

In [13]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, San Diego, California'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [14]:
coords = [ get_latlng(neighborhood) for neighborhood in san_df["Neighborhood"].tolist() ]

In [15]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
san_df['Latitude'] = df_coords['Latitude']
san_df['Longitude'] = df_coords['Longitude']
san_df.head()

,Neighborhood,Latitude,Longitude
0,Balboa Park,32.72734,-117.14788
1,Bankers Hill,32.72848,-117.16124
2,Barrio Logan,32.69467,-117.13808
3,Bay Ho,32.82779,-117.22133
4,Bay Park,32.79083,-117.19908


In [16]:
san_df.to_csv('SanDiego_neighborhood.csv')

**Dataframe can be directly imported to save time and computation**

In [4]:
address = 'San Diego, California'

geolocator = Nominatim(user_agent="ind_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Diego are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Diego are 32.7174209, -117.1627714.


In [5]:
neigh_df = pd.read_csv('SanDiego_neighborhood.csv')
neigh_df.drop(columns = ['Unnamed: 0'], inplace = True)
neigh_df.dropna(subset = ['Latitude','Longitude'], inplace = True)
neigh_df.shape

(120, 3)

In [6]:
map_san = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(neigh_df['Latitude'], neigh_df['Longitude'], neigh_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_san)  
    
map_san

In [42]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = '# your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:# your Foursquare Secret


In [19]:
LIMIT = 100
radius = 500

**The code cell below is used to extract out the venues from the json file recived from the foursquare API call and then return the top venues asked for.**

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [23]:
san_venues = getNearbyVenues(names=neigh_df['Neighborhood'],
                                   latitudes=neigh_df['Latitude'],
                                   longitudes=neigh_df['Longitude']
                                  )

Balboa Park
Bankers Hill
Barrio Logan
Bay Ho
Bay Park
Birdland
Black Mountain Ranch
Border
Burlingame
Carmel Mountain Ranch
Carmel Valley
City Heights
Clairemont
College Area
Del Mar Heights
Del Mar Mesa
Columbia
Core
Cortez Hill
East Village
Gaslamp Quarter
Horton Plaza
Little Italy
Marina
East Elliott
Egger Highlands
El Cerrito
Gateway
Golden Hill
Grant Hill
Harbor Island
Harborview
Hillcrest
Kearny Mesa
Kensington
La Jolla Village
Torrey Pines
Village of La Jolla
Lake Murray
Linda Vista
Logan Heights
Marston Hills
Memorial
Midtown
Mira Mesa
Miramar
Mission Beach
Mission Hills
Mission Valley
Civita
Civita
Morena
Allied Gardens
Del Cerro
Grantville
San Carlos
Nestor
Normal Heights
North City
North Park
North Clairemont
Oak Park
Ocean Beach
Ocean Crest
Ocean View Hills
Old Town
Otay Mesa
Otay Mesa West
Pacific Beach
Pacific Highlands Ranch
Palm City
La Playa
Loma Portal
Midway
Point Loma Heights
Roseville-Fleetridge
Sunset Cliffs
Wooded Area
Rancho Bernardo
Rancho Encantada
Rancho Peña

In [24]:
print(san_venues.shape)
san_venues.head()

(2246, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Balboa Park,32.72734,-117.14788,Spreckels Organ Pavilion,32.729621,-117.150393,Amphitheater
1,Balboa Park,32.72734,-117.14788,Balboa Park Activity Center,32.726507,-117.147743,Gym / Fitness Center
2,Balboa Park,32.72734,-117.14788,Japanese Friendship Garden,32.730054,-117.149985,Garden
3,Balboa Park,32.72734,-117.14788,The Tea Pavillion at the Japanese Friendship G...,32.730113,-117.149981,Tea Room
4,Balboa Park,32.72734,-117.14788,Balboa Park Visitor's Center,32.731143,-117.149919,Gift Shop


**The dataset is one-hot encoded in-order to calculate frequency of each type of venue so that it may be fed into the clustering algorithm**

In [25]:
san_onehot = pd.get_dummies(san_venues[['Venue Category']], prefix="", prefix_sep="")

# neighborhood column back to dataframe
san_onehot['Neighborhoods'] = san_venues['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(san_onehot.columns[-1:]) + list(san_onehot.columns[:-1])
san_onehot = san_onehot[fixed_columns]

print(san_onehot.shape)
san_onehot.head()

(2246, 297)


,Neighborhoods,ATM,Accessories Store,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Cruise,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Duty-free Shop,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoors & Recreation,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Public Bathroom,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roller Rink,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,State / Provincial Park,Steakhouse,Storage Facility,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Tour Provider,Toy / Game Store,Track,Trail,Train Station,Travel & Tra

In [26]:
san_grouped = san_onehot.groupby('Neighborhoods').mean().reset_index()
san_grouped.head()

,Neighborhoods,ATM,Accessories Store,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Cruise,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Duty-free Shop,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoors & Recreation,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Public Bathroom,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roller Rink,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,State / Provincial Park,Steakhouse,Storage Facility,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Tour Provider,Toy / Game Store,Track,Trail,Train Station,Travel & Tra

In [27]:
san_grouped.to_csv('San_neighborhood.csv')

**Now that the data has been extracted about the venues near the neighborhood we can work upon buiding the cluster and analysis.**

In [7]:
df_first = pd.read_csv('SanDiego_neighborhood.csv')
df_second = pd.read_csv('San_neighborhood.csv')

In [8]:
df_first.drop(columns = ['Unnamed: 0'], inplace = True)
df_first.head()

,Neighborhood,Latitude,Longitude
0,Balboa Park,32.72734,-117.14788
1,Bankers Hill,32.72848,-117.16124
2,Barrio Logan,32.69467,-117.13808
3,Bay Ho,32.82779,-117.22133
4,Bay Park,32.79083,-117.19908


In [9]:
df_second.drop(columns = ['Unnamed: 0'], inplace = True)
df_second.head()

,Neighborhoods,ATM,Accessories Store,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Casino,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Cruise,Cuban Restaurant,Cupcake Shop,Currency Exchange,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drive-in Theater,Dry Cleaner,Duty-free Shop,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Insurance Office,Intersection,Irish Pub,Island,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Meze Restaurant,Middle Eastern Restaurant,Military Base,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Motorsports Shop,Mountain,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Repair Shop,Outdoors & Recreation,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Pub,Public Bathroom,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roller Rink,Russian Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stables,State / Provincial Park,Steakhouse,Storage Facility,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Tour Provider,Toy / Game Store,Track,Trail,Train Station,Travel & Tra

In [11]:
print(df_first.shape)
print(df_second.shape)

(120, 3)
(116, 297)


**The difference in shape or rows is due to the fact that the API didn't have enough data for those Neighbourhoods which might implicate that either they are housing areas or simply the dataset lacks clarity**

**First we try to figure out clusters of Dataset to see if we can get some classes of clusters**

**Lets create a dataframe containing top 10 places of every neighborhood**

In [10]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [11]:
num_top_venues = 10
indicators = ['st','nd','rd']
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
San_sorted = pd.DataFrame(columns = columns)
San_sorted['Neighborhoods'] = df_second['Neighborhoods']
for ind in np.arange(df_second.shape[0]):
    San_sorted.iloc[ind, 1:] = return_most_common_venues(df_second.iloc[ind, :], num_top_venues)

San_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allied Gardens,Coffee Shop,Liquor Store,Café,Massage Studio,Fast Food Restaurant,Hawaiian Restaurant,Sandwich Place,Mexican Restaurant,Pet Store,Grocery Store
1,Alta Vista,Child Care Service,Liquor Store,Gym,Video Store,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant
2,Balboa Park,Food Truck,Museum,Coffee Shop,Garden,Gift Shop,Park,Donut Shop,Performing Arts Venue,Other Repair Shop,Sandwich Place
3,Bankers Hill,Pharmacy,American Restaurant,Coffee Shop,French Restaurant,South American Restaurant,Lounge,Gay Bar,Gastropub,Taco Place,Sushi Restaurant
4,Barrio Logan,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Record Shop,Nail Salon,Flea Market,Gas Station,Taco Place,Shopping Mall,Marijuana Dispensary


**Two step clustering - One overall clustering as done in previous modules**

In [12]:
kclusters = 6

san_grouped_clustering = df_second.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(san_grouped_clustering)

In [13]:
San_sorted['Cluster Label'] = kmeans.labels_ #joining the labels to the existing table
San_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
0,Allied Gardens,Coffee Shop,Liquor Store,Café,Massage Studio,Fast Food Restaurant,Hawaiian Restaurant,Sandwich Place,Mexican Restaurant,Pet Store,Grocery Store,0
1,Alta Vista,Child Care Service,Liquor Store,Gym,Video Store,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,0
2,Balboa Park,Food Truck,Museum,Coffee Shop,Garden,Gift Shop,Park,Donut Shop,Performing Arts Venue,Other Repair Shop,Sandwich Place,0
3,Bankers Hill,Pharmacy,American Restaurant,Coffee Shop,French Restaurant,South American Restaurant,Lounge,Gay Bar,Gastropub,Taco Place,Sushi Restaurant,0
4,Barrio Logan,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Record Shop,Nail Salon,Flea Market,Gas Station,Taco Place,Shopping Mall,Marijuana Dispensary,0


In [14]:
San_sorted.shape

(116, 12)

In [15]:
san_merged = df_first
san_merged.rename(columns = {"Neighborhood" : "Neighborhoods"},inplace = True)
san_merged = san_merged.join(San_sorted.set_index('Neighborhoods'), on='Neighborhoods')
san_merged.head() #joining the tables inorder to get all the columns

,Neighborhoods,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
0,Balboa Park,32.72734,-117.14788,Food Truck,Museum,Coffee Shop,Garden,Gift Shop,Park,Donut Shop,Performing Arts Venue,Other Repair Shop,Sandwich Place,0.0
1,Bankers Hill,32.72848,-117.16124,Pharmacy,American Restaurant,Coffee Shop,French Restaurant,South American Restaurant,Lounge,Gay Bar,Gastropub,Taco Place,Sushi Restaurant,0.0
2,Barrio Logan,32.69467,-117.13808,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Record Shop,Nail Salon,Flea Market,Gas Station,Taco Place,Shopping Mall,Marijuana Dispensary,0.0
3,Bay Ho,32.82779,-117.22133,Health & Beauty Service,Pool,Trail,Pizza Place,Asian Restaurant,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,0.0
4,Bay Park,32.79083,-117.19908,Massage Studio,Gas Station,Convenience Store,Hotel,Coffee Shop,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,0.0


In [ ]:
#san_merged.drop(columns = ['Unnamed: 0'], inplace = True)

In [18]:
san_merged.dropna(subset = ['Cluster Label'], inplace = True)
san_merged.shape

(116, 14)

In [ ]:
san_merged = san_merged.astype({'Cluster Label': int})
san_merged.head()

In [20]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(san_merged['Latitude'], san_merged['Longitude'], san_merged['Neighborhoods'], san_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**cluster 1 --- Determination of type of area (commercial, housing, etc)**

In [16]:
san_merged.loc[san_merged['Cluster Label'] == 0, san_merged.columns[[1] + list(range(3, san_merged.shape[1]))]].head()

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
0,32.72734,Food Truck,Museum,Coffee Shop,Garden,Gift Shop,Park,Donut Shop,Performing Arts Venue,Other Repair Shop,Sandwich Place,0.0
1,32.72848,Pharmacy,American Restaurant,Coffee Shop,French Restaurant,South American Restaurant,Lounge,Gay Bar,Gastropub,Taco Place,Sushi Restaurant,0.0
2,32.69467,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Record Shop,Nail Salon,Flea Market,Gas Station,Taco Place,Shopping Mall,Marijuana Dispensary,0.0
3,32.82779,Health & Beauty Service,Pool,Trail,Pizza Place,Asian Restaurant,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,0.0
4,32.79083,Massage Studio,Gas Station,Convenience Store,Hotel,Coffee Shop,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,0.0


**Cluster 2**

In [50]:
san_merged.loc[san_merged['Cluster Label'] == 1, san_merged.columns[[1] + list(range(3, san_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
76,32.71452,Record Shop,Ethiopian Restaurant,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,Flea Market,Flower Shop,1


**Cluster 3**

In [17]:
san_merged.loc[san_merged['Cluster Label'] == 2, san_merged.columns[[1] + list(range(3, san_merged.shape[1]))]].head()

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
18,33.152584,Park,Home Service,Skate Park,Food Truck,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Ethiopian Restaurant,2.0
24,32.742976,Park,Baseball Field,Fountain,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Event Space,2.0
52,32.787620,ATM,Bagel Shop,Park,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,2.0
78,32.927180,Park,Home Service,Fishing Spot,Event Space,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Yoga Studio,2.0
86,32.929990,Lawyer,Park,Yoga Studio,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,2.0


**Cluster 4**

In [52]:
san_merged.loc[san_merged['Cluster Label'] == 3, san_merged.columns[[1] + list(range(3, san_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
98,32.71551,Recording Studio,Ethiopian Restaurant,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,Flea Market,Flower Shop,3


**Cluster 5**

In [18]:
san_merged.loc[san_merged['Cluster Label'] == 4, san_merged.columns[[1] + list(range(3, san_merged.shape[1]))]].head()

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
8,32.733756,Bookstore,Pizza Place,Arts & Crafts Store,Liquor Store,Nail Salon,Flower Shop,Mexican Restaurant,Fondue Restaurant,Fast Food Restaurant,Filipino Restaurant,4.0
12,32.834560,Mexican Restaurant,Ramen Restaurant,Chinese Restaurant,Recreation Center,Liquor Store,Mobile Phone Shop,Trail,Middle Eastern Restaurant,Donut Shop,Fishing Spot,4.0
27,32.725261,Convenience Store,Mexican Restaurant,Pizza Place,Market,Food Court,Food & Drink Shop,Food,Food Truck,Fondue Restaurant,Flower Shop,4.0
42,32.695980,Mexican Restaurant,Deli / Bodega,Vape Store,Burrito Place,Fast Food Restaurant,Harbor / Marina,Automotive Shop,Laundry Service,Laundromat,Fried Chicken Joint,4.0
55,32.576490,Gas Station,Mexican Restaurant,Breakfast Spot,Intersection,Fast Food Restaurant,Motel,Furniture / Home Store,Pizza Place,Seafood Restaurant,Flea Market,4.0


In [54]:
san_merged.loc[san_merged['Cluster Label'] == 5, san_merged.columns[[1] + list(range(3, san_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label
15,32.94257,Trail,Yoga Studio,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,Flower Shop,5


*San diego is known for its lively nature which is quite evident since the cluster 0 identifies areas with most visiting areas such as tons of restaurants, malls, shops and other things.*

*Clustering helps us understand what areas are commercial and which are not that commercial*

In [21]:
df_mall = df_second[["Neighborhoods","Shopping Mall"]]
df_mall.head()

,Neighborhoods,Shopping Mall
0,Allied Gardens,0.000000
1,Alta Vista,0.000000
2,Balboa Park,0.000000
3,Bankers Hill,0.000000
4,Barrio Logan,0.045455


In [22]:
kclusters = 3

san_clustering = df_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans2 = KMeans(n_clusters=kclusters, random_state=0).fit(san_clustering)

# check cluster labels generated for each row in the dataframe
kmeans2.labels_[0:10]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int32)

In [28]:
df_mall['Shopping Labels'] = kmeans2.labels_
df_mall.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Neighborhoods,Shopping Labels
0,Allied Gardens,0
1,Alta Vista,0
2,Balboa Park,0
3,Bankers Hill,0
4,Barrio Logan,1


In [29]:
san_merged = san_merged.join(df_mall.set_index('Neighborhoods'), on='Neighborhoods')
san_merged.head()

,Neighborhoods,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label,Shopping Labels
0,Balboa Park,32.72734,-117.14788,Food Truck,Museum,Coffee Shop,Garden,Gift Shop,Park,Donut Shop,Performing Arts Venue,Other Repair Shop,Sandwich Place,0,0
1,Bankers Hill,32.72848,-117.16124,Pharmacy,American Restaurant,Coffee Shop,French Restaurant,South American Restaurant,Lounge,Gay Bar,Gastropub,Taco Place,Sushi Restaurant,0,0
2,Barrio Logan,32.69467,-117.13808,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Record Shop,Nail Salon,Flea Market,Gas Station,Taco Place,Shopping Mall,Marijuana Dispensary,0,1
3,Bay Ho,32.82779,-117.22133,Health & Beauty Service,Pool,Trail,Pizza Place,Asian Restaurant,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,0,0
4,Bay Park,32.79083,-117.19908,Massage Studio,Gas Station,Convenience Store,Hotel,Coffee Shop,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,0,0


In [30]:
san_merged.to_csv('Final_data_capstone.csv')

So basically the whole point of two different clustering is to find out via first clustering those areas where their is chance of successful shopping complex and the other clustering is to identify areas where there are less shopping complex than the required or in other words that location may stand a chance to stand and excel.

So we can see clearly that the three different shopping labels all lie within the cluster 0 which indicates that shopping mall constitute a part of that area

Extracting area with the cluster 0 as the first classification

In [27]:
san_merged = pd.read_csv('Final_data_capstone.csv')

In [28]:
new_df = san_merged[san_merged['Cluster Label'] == 0]
new_df.shape

(89, 16)

In [29]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(new_df['Latitude'], new_df['Longitude'], new_df['Neighborhoods'], new_df['Shopping Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

shopping cluster 1 locations

In [24]:
new_df.loc[new_df['Shopping Labels'] == 0, new_df.columns[[1] + list(range(3, new_df.shape[1]))]]

,Neighborhoods,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label,Shopping Labels
0,Balboa Park,-117.147880,Food Truck,Museum,Coffee Shop,Garden,Gift Shop,Park,Donut Shop,Performing Arts Venue,Other Repair Shop,Sandwich Place,0,0
1,Bankers Hill,-117.161240,Pharmacy,American Restaurant,Coffee Shop,French Restaurant,South American Restaurant,Lounge,Gay Bar,Gastropub,Taco Place,Sushi Restaurant,0,0
3,Bay Ho,-117.221330,Health & Beauty Service,Pool,Trail,Pizza Place,Asian Restaurant,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,0,0
4,Bay Park,-117.199080,Massage Studio,Gas Station,Convenience Store,Hotel,Coffee Shop,Flea Market,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,0,0
5,Birdland,-117.154310,Burrito Place,Gas Station,Convenience Store,Video Store,Flower Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,0,0
6,Black Mountain Ranch,-117.145150,Golf Course,Yoga Studio,Flower Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fishing Spot,Flea Market,Fondue Restaurant,0,0
7,Border,-117.033860,Fast Food Restaurant,Border Crossing,Clothing Store,Shoe Store,Bus Station,Mobile Phone Shop,Music Store,Fishing Spot,Boutique,Light Rail Station,0,0
9,Carmel Mountain Ranch,-117.085119,Fast Food Restaurant,Ice Cream Shop,Italian Restaurant,Sandwich Place,Mexican Restaurant,Donut Shop,Cosmetics Shop,Chinese Restaurant,Supplement Shop,Miscellaneous Shop,0,0
11,City Heights,-117.107150,Liquor Store,Dive Bar,Gym / Fitness Center,Fast Food Restaurant,Mexican Restaurant,Sandwich Place,Pet Store,Latin American Restaurant,Asian Restaurant,Discount Store,0,0
13,College Area,-116.831200,Casino,Food Court,Steakhouse,Mexican Restaurant,Deli / Bodega,Performing Arts Venue,Clothing Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,0,0


cluster 2

In [25]:
new_df.loc[new_df['Shopping Labels'] == 1, new_df.columns[[1] + list(range(3, new_df.shape[1]))]]

,Neighborhoods,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label,Shopping Labels
2,Barrio Logan,-117.138080,Fast Food Restaurant,Coffee Shop,Mexican Restaurant,Record Shop,Nail Salon,Flea Market,Gas Station,Taco Place,Shopping Mall,Marijuana Dispensary,0,1
10,Carmel Valley,-117.230860,Bank,Martial Arts Dojo,Gym,Grocery Store,Coffee Shop,Gas Station,Sandwich Place,Taco Place,Sushi Restaurant,Dance Studio,0,1
33,Kearny Mesa,-117.149620,Rental Car Location,Japanese Restaurant,Korean Restaurant,Fast Food Restaurant,Sushi Restaurant,Yoga Studio,Bookstore,Business Service,Café,Shopping Mall,0,1
35,La Jolla Village,-117.233220,Furniture / Home Store,Shopping Mall,Resort,Clothing Store,Mexican Restaurant,Coffee Shop,Movie Theater,Gym,Pet Store,Bank,0,1
37,Village of La Jolla,-117.233220,Furniture / Home Store,Shopping Mall,Resort,Clothing Store,Mexican Restaurant,Coffee Shop,Movie Theater,Gym,Pet Store,Bank,0,1
39,Linda Vista,-117.171350,Vietnamese Restaurant,Thai Restaurant,ATM,Grocery Store,Massage Studio,Liquor Store,Farmers Market,Mexican Restaurant,Donut Shop,Pharmacy,0,1
44,Mira Mesa,-117.150720,Mobile Phone Shop,Mexican Restaurant,Sandwich Place,Dessert Shop,Department Store,Grocery Store,Deli / Bodega,Liquor Store,Tea Room,Restaurant,0,1
49,Civita,-117.147748,Furniture / Home Store,Gym / Fitness Center,Dog Run,Clothing Store,Paper / Office Supplies Store,Sandwich Place,Salon / Barbershop,Park,Fast Food Restaurant,Mexican Restaurant,0,1
77,Rancho Peñasquitos,-117.128460,Fried Chicken Joint,Auto Workshop,Fast Food Restaurant,Chinese Restaurant,Eye Doctor,Shopping Mall,Sandwich Place,Gym / Fitness Center,Spa,Gastropub,0,1


cluster 3

In [26]:
new_df.loc[new_df['Shopping Labels'] == 2, new_df.columns[[1] + list(range(3, new_df.shape[1]))]]

,Neighborhoods,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Label,Shopping Labels
14,Del Mar Heights,-117.26087,Hotel,Yoga Studio,Surf Spot,Beach,Park,Shopping Mall,American Restaurant,Food Court,Food & Drink Shop,Food,0,2


Observation - Shopping Label 0 - indicates that the area has very less proportion of shopping complex and has other features.
Shopping Label 1 - indicates that the area has moderate amount of shopping complex and that they are flourishing
Shopping Label 2 - indicates that the area has good shopping complex and it is known for them. This tells us the areas which are commercial and where you might find a place for your business. The labels have a deep meaning which depends upon the consumer. So the consumer might be afraid of competition or he may find some cheap land in cluster 0 of shopping that is where there are low number of complex. This decision depends upon the mindset you have and the assets. An apt choice would be build in an area where there are some shopping complex which are there to tell us that the area recognizes them and that they are flourishing. The third kind is for people who are more confident or risk takers because they already know that the cluster 2 is famous for shopping and if we are able to build something extraordinary then we might end up stealing away a lot of consumers which have been built by the other complex. This area is sensitive to competition and ups and down.